In [ ]:
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')   # The object 'warnings' is used to call the method 'filterwarnings' and ignore the warnings

In [ ]:
# import the file Rental_Prices(RTB)
df_rent= pd.read_csv('housing df.csv')

In [ ]:
df_rent.head()

In [ ]:
df_rent.shape

In [ ]:
df_rent.describe()

In [ ]:
df_rent.isnull().sum()

In [ ]:
df_rent.drop(df_rent.loc[df_rent['Year']<2023].index, inplace=True)

In [ ]:
df_rent

In [ ]:
df_rent.dropna(axis=0,inplace=True)

In [ ]:
df_rent.shape

In [ ]:
df_rent.isnull().sum()

In [ ]:
df_rent.info()

In [ ]:
df_rent.drop(["STATISTIC", "STATISTIC Label", "TLIST(A1)","C02970V03592", "C02969V03591", "C03004V03625"] , axis=1,inplace=True)

In [ ]:
df_rent.info()

In [ ]:
mean_value = df_rent["VALUE"].mean()

In [ ]:
# Plot the distribution with a KDE
sns.displot(df_rent["VALUE"], kde=True)
plt.axvline(mean_value, color='red', label=f'Mean: {mean_value:.2f}')
plt.legend()
plt.show()

In [ ]:
df_rent["Location"].unique

In [ ]:
df_dublin = df_rent[df_rent["Location"].str.contains("Dublin")]

In [ ]:
df_dublin["Location"].unique